In [ ]:
# !pip install tbats

In [ ]:
import pandas as pd
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from tbats import TBATS
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
# import numpy as np
import warnings
from sklearn.preprocessing import MinMaxScaler

# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.api import VAR

# Ignorar todos los warnings
warnings.filterwarnings("ignore")

In [ ]:
# Cargamos los datos y fijamos la columna de fechas
date_col = 'date'
df = pd.read_excel('dataframe.xlsx')
df[date_col] = pd.to_datetime(df[date_col])
df.set_index(date_col, inplace=True)


In [ ]:
factor = (df['CONSUMER PRICE INDEX'] / df['CONSUMER PRICE INDEX'].shift(1)).mean(skipna=True)
# df['CONSUMER PRICE INDEX'] = df['CONSUMER PRICE INDEX'].fillna(method='ffill') * factor


In [ ]:
scaler = MinMaxScaler()

# Normalizar el DataFrame excepto la columna 'date'
columns_to_normalize = df.columns.difference(['date','USREC'])  # Excluir la columna 'date'
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Suponiendo que df contiene tu DataFrame original con los datos
X = df.drop('USREC', axis=1)
y = df['USREC']

In [ ]:
df.tail(5)

In [ ]:

def grid_search_var(data, maxlags):
    best_aic = float("inf")
    best_order = None
    best_model = None
    
    for lags in range(1, maxlags+1):
        model = VAR(data)
        result = model.fit(lags)
        if result.aic < best_aic:
            best_aic = result.aic
            best_order = lags
            best_model = result
    
    return best_model, best_order, best_aic

def predict_next_periods(df, periods=12, maxlags=5):
    # Hacer la búsqueda en rejilla para encontrar el mejor modelo VAR
    best_model, best_order, best_aic = grid_search_var(df, maxlags)
    
    print(f"Best AIC: {best_aic} with order: {best_order}")
    
    # Hacer predicciones
    forecast = best_model.forecast(df.values[-best_order:], steps=periods)
    
    # Obtener la fecha de inicio de la predicción
    start_date = df.index[-1] + pd.DateOffset(months=1)
    
    # Generar índice de fechas para las predicciones
    forecast_index = pd.date_range(start=start_date, periods=periods, freq='MS')
    
    # Convertir las predicciones a un DataFrame
    forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=df.columns)
    
    return forecast_df, best_model

def calculate_training_metric(model, df):
    # Obtener las predicciones dentro de la muestra
    in_sample_forecast = model.fittedvalues
    
    # Calcular el Error Cuadrático Medio (MSE) para cada columna
    mse_dict = {}
    for col in df.columns:
        mse_dict[col] = mean_squared_error(df[col].iloc[model.k_ar:], in_sample_forecast[col])
    
    return mse_dict

def predict_specific_periods(df, periods_list, maxlags=5):
    result_dict = {}
    
    # Obtener la fecha inicial de predicción
    start_date = df.index[-1] + pd.DateOffset(months=1)
    
    for periods in periods_list:
        # Calcular la fecha de predicción específica
        prediction_date = start_date + pd.DateOffset(months=periods)
        
        # Predecir los próximos períodos
        predictions, best_model = predict_next_periods(df, periods=periods, maxlags=maxlags)
        
        # Guardar la predicción en el diccionario de resultados
        result_dict[prediction_date] = predictions.iloc[-1]  # Obtener la última fila que corresponde a la predicción final del período
    
    # Crear un DataFrame con las predicciones para los períodos específicos
    result_df = pd.DataFrame(result_dict).T
    result_df.index.name = 'Fecha'  # Nombre del índice
    
    return result_df



# Lista de períodos a predecir
periods_list = [12, 24, 36]

# Predecir los períodos especificados y obtener el DataFrame resultante
predictions_df = predict_specific_periods(df, periods_list, maxlags=5)

# Imprimir el DataFrame resultante con las fechas exactas en las filas
print("Predicciones para 12, 24 y 36 meses:")
print(predictions_df)

In [ ]:
predictions_df

In [ ]:
predictions_df = predictions_df.iloc[:, :7]

In [ ]:
predictions_df

In [ ]:
# predictions_df = predictions_df.drop('USREC', axis = 1)
predictions_df.to_excel('predictions_years.xlsx')